In [21]:
import torch as t
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt

# Measured Data for x1 and p 
x1_data = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
p_data = [28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.7, 17.5]

# saturation pressure = 10.0**((a[1] - (a[2] / (T + a[3]))))
psat_wat = 10.0**(8.07131-1730.63/(20.0+233.426))
psat_diox = 10.0**(7.43155-1554.679/(20.0+240.337))

#defining a loss func.
def loss(a): 
    loss_value = 0.0
    for i in range(11):
        x1 = x1_data[i]
        p = p_data[i]
        p_new = x1*psat_wat*t.exp(a[0]*(a[1]*(1-x1)/(a[0]*x1+a[1]*(1-x1)))**2) + (1-x1)*psat_diox*t.exp(a[1]*( a[0]*x1/( a[0]*x1+a[1]*(1-x1) ) )**2)
        loss_value = loss_value + (p_new-p)**2
    return loss_value
#assume error as 1
error = 1
#initialize A12 and A21 
A = Variable(t.tensor([1.0, 1.0]), requires_grad=True)

# gradient decent
while error >= 0.1:
    loss(A).backward()
    error = t.linalg.norm(A.grad)
    # Perform line search n ensure smaller step size
    s=.1  # intialize step size
    while loss(A-s*A.grad) > loss(A):
        s=.3*s
    with t.no_grad():       
        A -= s*A.grad
        A.grad.zero_()

print(' Optimized value of A12, A21 =', A)
print(' value of loss function =', loss(A).data.numpy())

from math import exp
A = [1.9582, 1.6893]
print(' COMPARISON TABLE')
print('Given data        Calc. data            error')
for i in range(11):
    x1 = x1_data[i]
    p_new = x1*psat_wat*exp(A[0]*(A[1]*(1-x1)/(A[0]*x1+A[1]*(1-x1)))**2) + (1-x1)*psat_diox*exp(A[1]*( A[0]*x1/( A[0]*x1+A[1]*(1-x1) ) )**2)
    print(p_data[i], ' ------------ ',round(p_new,3) ,'------------', round((p_new-p_data[i])/p_data[i],4))



 Optimized value of A12, A21 = tensor([1.9581, 1.6894], requires_grad=True)
 value of loss function = 0.6702018
 COMPARISON TABLE
Given data        Calc. data            error
28.1  ------------  28.824 ------------ 0.0258
34.4  ------------  34.643 ------------ 0.0071
36.7  ------------  36.452 ------------ -0.0068
36.9  ------------  36.867 ------------ -0.0009
36.8  ------------  36.873 ------------ 0.002
36.7  ------------  36.749 ------------ 0.0013
36.5  ------------  36.39 ------------ -0.003
35.4  ------------  35.385 ------------ -0.0004
32.9  ------------  32.948 ------------ 0.0015
27.7  ------------  27.731 ------------ 0.0011
17.5  ------------  17.473 ------------ -0.0015
